In [1]:
pip install yahist

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import uproot
%matplotlib inline
from coffea import hist
#from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import cloudpickle
import gzip
import time
import os
from yahist import Hist1D, Hist2D       

In [3]:
class Processor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "") 
        pt_axis = hist.Bin("pt", " [GeV]", [10., 15., 20., 25., 35., 50., 70., 90.])
        eta_mu_axis= hist.Bin("eta", "", [0., 1.2, 2.1, 2.4])
        eta_el_axis= hist.Bin("eta", "", [0., 0.8, 1.479, 2.5])
        
        self._accumulator = processor.dict_accumulator({
            'Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Tight_Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis),
            'Tight_Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis)

        })
    
    @property
    def accumulator(self):
        
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        dataset = events.metadata["dataset"]
        
        # Skim Muon, Electron, and Jet
        Muon = events.Muon
        Muon = Muon[( Muon.pt >=10 ) &
                           (np.abs(Muon.eta) <= 2.4) &
                           (np.abs(Muon.dxy) <= 0.05) &
                           (np.abs(Muon.dz) <= 0.1) &
                           (np.abs(Muon.sip3d) < 4) &
                           (Muon.looseId==1) &
                           (Muon.ptErr/Muon.pt < 0.2) &
                           (Muon.mediumId==1) ]
        
        Electron = events.Electron
        Electron = Electron[( Electron.pt >10 ) &  #  (Electron.isTriggerSafeNolso) 
                           (np.abs(Electron.eta+Electron.deltaEtaSC ) < 2.4) &
                            (Electron.convVeto)&
                            (Electron.lostHits==0) &
                            (Electron.tightCharge==2) &
                            (np.abs(Electron.dz) < 0.1) &
                           (np.abs(Electron.dxy) < 0.05) &
                           (np.abs(Electron.sip3d) < 4) ]
        Jet = events.Jet
        Jet = Jet[( Jet.pt >25 ) &  (np.abs(Jet.eta ) < 2.4) ]

        
        def delta_phi(first, second):
            return np.arccos(np.cos(first.phi - second.phi))

        def delta_r2(first, second):
            return (first.eta - second.eta) ** 2 + delta_phi(first, second) ** 2

        def match(first, second, deltaRCut=0.4):
            drCut2 = deltaRCut**2
            combs = ak.cartesian([first, second], nested=True)
            return ak.any((delta_r2(combs['0'], combs['1'])<drCut2), axis=2)    
        
        def mt(pt1, phi1, pt2, phi2):    #transverse mass
            return np.sqrt( 2*pt1*pt2 * (1 - np.cos(phi1-phi2)) )
        
        # Definition of Tight/Loose Muon and Electron
        Tight_Muon = Muon[( Muon.pt >20 ) &
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.tightId==1) &
                          (Muon.miniPFRelIso_all<0.16)
                         ]
        Tight_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                       #  (Electron.tightId==1) &
                          (Electron.miniPFRelIso_all<0.12)
                         ]
        
        Loose_Muon = Muon[( Muon.pt >20 )&
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.looseId==1) &
                          (Muon.miniPFRelIso_all<0.4)
                         ]        
        
        Loose_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                        # (Electron.looseId==1) &
                          (Electron.miniPFRelIso_all<0.4)
                         ]
        MET = events.MET   
        

        control_region_Muon = events[( ak.num(Loose_Muon)==1) &(ak.num(Loose_Electron)==0)
                                    & (ak.num(Jet[~match(Jet, Muon, deltaRCut=1.0)])>=1) & ( MET.pt<20) 
                                    & ak.any(mt(Muon.pt,Muon.phi, MET.pt, MET.phi)<20, axis=1)]

        
        tight_muon= control_region_Muon.Muon[(control_region_Muon.Muon.pt >20 ) &
                          (np.abs(control_region_Muon.Muon.eta) < 2.4) &
                         (control_region_Muon.Muon.tightId==1) &
                          (control_region_Muon.Muon.miniPFRelIso_all<0.16)
                         ]
        output['Tight_Muon'].fill(dataset=dataset, pt=ak.flatten(tight_muon.pt), eta=ak.flatten(tight_muon.eta))
        output['Muon'].fill(dataset=dataset, pt=ak.flatten(control_region_Muon.Muon.pt), eta=ak.flatten(control_region_Muon.Muon.eta))
        return output
    def postprocess(self, accumulator):
        return accumulator

In [4]:
QCD = ['root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/E014301D-D1F4-CD49-A9E9-5D99D79B00D1.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/410ACFE6-44C7-C943-88A7-298EA225B8F2.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/6DE7C6FC-3C5B-FE42-88B3-E2BC4E45591C.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/C134C680-0D29-7848-A39D-7B6164CD6CA5.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/256C855D-80AE-604F-98C5-669F545944F7.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/EE1DDC6E-50AB-3544-B973-CDDEB1BC7B8D.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/0C5F5AD3-DCF2-1845-B933-2B1BE5D9316F.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/9154517F-5214-394F-9999-2AA16E72FBAD.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/E3B7F0DE-6BD0-144F-8324-81F362BD3435.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/DED1443C-8E4D-AA4E-AFAA-8CA9E5985D64.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/6C12E976-D722-E34C-9A4F-F50CA991E855.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/19CCFAD5-B882-4148-82B5-7697A38148E2.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/BF42D99A-6B96-EA42-8179-A6E75EAC4D6F.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/B2F35080-4200-7C4B-B808-24EA296BD632.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/2DAEA590-FA3F-F542-B8CA-3ACD4CB4FB49.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/3A766B34-8AA1-4D44-9DCC-3B057A5E4BF4.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/FCBC9614-C758-F247-9190-20A020434CFE.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/270000/39CD784C-7C80-6740-BF7C-5C6966F20A44.root',
'root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20toInf_MuEnrichedPt15_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/70000/5622B823-AA50-AC43-87B2-82E30393A6A2.root']

In [ ]:
#fileset = {'SingleMu' : ["root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/60000/30AED9F5-92AD-DD46-A547-61765FD6B596.root"]}
fileset = {'SingleMu' :QCD}

from dask.distributed import Client
client = Client("tls://localhost:8786")

def unique(filename):
    file, ext = os.path.splitext(filename)
    counter = 0
    while os.path.exists(filename):
        counter += 1
        filename = file + str(counter) + ext
    return filename

tstart = time.time()
chunksize = 250000

from dask.distributed import performance_report
fname = unique("dask-report_chunksize=" + str(chunksize/1000) + "K.html")
with performance_report(filename=fname):
    output = processor.run_uproot_job(fileset,
                                     treename='Events',
                                     processor_instance=Processor(),
                                     #executor=processor.futures_executor,
                                     executor=processor.dask_executor,
                                     executor_args={"schema": NanoAODSchema, 'client': client, 'savemetrics': True},
                                     chunksize = chunksize)

[##################################      ] | 86% Completed |  5min 20.6ss

In [ ]:
dt = time.time() - tstart
outname = 'QCD'
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')
print("Events / s / thread: {:,.0f}".format(output[1]['entries'] / output[1]['processtime']))
print("Events / s: {:,.0f}".format(output[1]['entries'] / dt))

In [ ]:
output

### Plotting
https://github.com/cmstas/FTAnalysis/blob/run2/analysis/fakes/derivation/ScanChain_fast.C#L161

In [ ]:
import pickle
path = 'histos/QCD.pkl.gz'
print('Opening path: ', path)
hists = {}
with gzip.open(path) as fin:
    QCD= pickle.load(fin)

In [ ]:
QCD

In [ ]:
Loose_Muon = Hist2D.from_bincounts(
    QCD[0]['Muon'].sum().values()[('SingleMu',)].T,
    (QCD[0]['Muon'].axis('pt').edges(), QCD[0]['Muon'].axis('eta').edges())
)

In [ ]:
Loose_Muon

In [ ]:
Tight_Muon = Hist2D.from_bincounts(
    QCD[0]['Tight_Muon'].sum().values()[('SingleMu',)].T,
    (QCD[0]['Tight_Muon'].axis('pt').edges(), QCD[0]['Tight_Muon'].axis('eta').edges())
)

In [ ]:
Tight_Muon

In [ ]:
Tight_Muon.divide(Loose_Muon)